## Liste des modèles à tester (et des hyper-paramètres correspondants)
* régression linéaire régularisée (*elastic net*)
    * équilibre entre la régularisation l1 et la régularisation l2
    * intensité de la régularisation


* machine à vecteur de support à noyau pour la régression (k-SVR)
    * C : pénalité pour la mauvaise classification (absent pour la régression ?)
    * kernel : nature du noyau (linéaire, gaussien, etc.)
    * gamma : paramètre pour le noyau gaussien


* forêt aléatoire pour la régression (*random forest regressor*)
    * max_features : nombre de variables à prendre en compte à chaque noeud
    * n_estimators : nombre d'arbres de décisions à entraîner
    

* AdaBoost
    * n_estimators : nombre d'arbres de décisions à entraîner
    * learning_rate : contrôle la vitesse d'apprentissage via la contribution de chaque modèle à la pondération
    * loss : définition de la fonction de perte (linear, square, exponential)
    
    
* k-plus proches voisins pour la régression (k-NN regressor)
    * k : nombre de voisins considérés
    
    
* réseau de neurones

## Chargement des données nettoyées

In [17]:
# Load libraries
import numpy as np
import pandas as pd

In [30]:
# Load binary file with cleaned data
data = pd.read_feather("p4_data4.ft")

# Display head of the dataframe
data.head()

,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions
0,Multifamily LR (1-4),Low-Rise Multifamily,DOWNTOWN,1900.0,1.0,4.0,48383.0,0.0,48383.0,Multifamily Housing,38172.000000,75.0,2953338.0,112.06
1,Multifamily LR (1-4),Low-Rise Multifamily,DELRIDGE,2007.0,1.0,4.0,52134.0,0.0,52134.0,Multifamily Housing,52134.000000,85.0,1212551.0,8.45
2,NonResidential,Retail Store,DOWNTOWN,1989.0,1.0,4.0,111077.0,0.0,111077.0,Retail Store,101528.658703,91.0,9898724.0,69.01
3,NonResidential,Small- and Mid-Sized Office,DOWNTOWN,1906.0,1.0,6.0,98370.0,25920.0,72450.0,Office,98370.000000,45.0,6525887.0,47.24
4,NonResidential,Large Office,LAKE UNION,1947.0,1.0,4.0,193788.0,37854.0,155934.0,Office,138672.000000,59.0,16760217.0,116.84


# Modélisation
Nous cherchons à élaborer un modèle de prédiction des émissions de CO2 et de la consommation totale d'énergie à partir d'un jeu de données.

Il s'agit donc d'un problème « *machine learning* » de régression (donc un problème d'apprentissage supervisé).

## Séparation des variables explicatives (features) et des étiquettes (targets)
Il y a deux variables à estimer :
* 'TotalGHGEmissions': *The total amount of greenhouse gas emissions, including carbon dioxide, methane, and nitrous oxide gases released into the atmosphere as a result of energy consumption at the property, measured in metric tons of carbon dioxide equivalent.*
* 'SiteEnergyUse(kBtu)': *The annual amount of energy consumed by the property from all sources of energy.*

## Séparation du jeu de données étiquettées et du jeu de données non-étiquettées
Nous avons vu précédemment que certaines données (en fait, une seule) n'ont pas de valeurs renseignées pour la valeur cible.
Nous allons mettre à part ces lignes pour un traitement avec notre modèle.

In [31]:
def missing_target(dataframe, target):
    """This function takes a dataframe and target feature, and returns
    a view of the dataframe containing the rows where the target value is missing.
    It also remove these rows from the original dataframe."""
    
    # Mask for rows where the target value is missing
    mask = dataframe[target].isnull()
    
    # Applying the mask to the dataframe and saving it as a copy
    result = dataframe[mask].copy()
    
    # Deleting the rows with missing target from the dataframe
    dataframe.dropna(subset=[target], inplace=True)
    
    # Returning a dataframe with rows where target is missing
    return result

In [36]:
target_cols = ['SiteEnergyUse(kBtu)','TotalGHGEmissions']
dataframe = data

# concatenating rows with missing values on any target
for target in target_cols:
    production_set = pd.concat([
        production_set,
        missing_target(dataframe, target)
    ])

# dropping duplicates rows (missing several targets)
production_set = production_set.drop_duplicates(keep='first')

# Display the 'production set' on which to apply the ML model
production_set

,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions
36,SPS-District K-12,K-12 School,NORTHWEST,1953.0,1.000000,1.0,110830.0,0.0,110830.0,K-12 School,110830.000000,67.676080,NaN,NaN
65,Multifamily LR (1-4),Low-Rise Multifamily,MAGNOLIA / QUEEN ANNE,1957.0,0.705243,4.0,23636.0,0.0,23636.0,Low-Rise Multifamily,18758.287287,67.396583,NaN,NaN
900,Multifamily LR (1-4),Low-Rise Multifamily,NORTHEAST,1977.0,1.000000,3.0,33166.0,0.0,33166.0,Low-Rise Multifamily,27778.915621,67.433917,NaN,NaN
120,Multifamily LR (1-4),Low-Rise Multifamily,EAST,2015.0,1.000000,4.0,36685.0,8254.0,28431.0,Low-Rise Multifamily,25186.251336,67.813506,820220.125,NaN


Nous voyons que nous n'avons ici qu'un seul bâtiment pour lequel les deux valeurs cibles ne sont pas renseignées. Nous utiliserons ce jeu de donner pour valider le flux de travail à l'issue de la modélisation.

### Séparation des variables explicatives et des étiquettes

In [37]:
# Features : all data DataFrame, except targets (2 last columns)
X = data.iloc[:, 0:-2]

# Targets : projection on multiple columns, by name
y = data[['SiteEnergyUse(kBtu)','TotalGHGEmissions']]

In [38]:
X.head()

,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,ENERGYSTARScore
0,Multifamily LR (1-4),Low-Rise Multifamily,DOWNTOWN,1900.0,1.0,4.0,48383.0,0.0,48383.0,Multifamily Housing,38172.000000,75.0
1,Multifamily LR (1-4),Low-Rise Multifamily,DELRIDGE,2007.0,1.0,4.0,52134.0,0.0,52134.0,Multifamily Housing,52134.000000,85.0
2,NonResidential,Retail Store,DOWNTOWN,1989.0,1.0,4.0,111077.0,0.0,111077.0,Retail Store,101528.658703,91.0
3,NonResidential,Small- and Mid-Sized Office,DOWNTOWN,1906.0,1.0,6.0,98370.0,25920.0,72450.0,Office,98370.000000,45.0
4,NonResidential,Large Office,LAKE UNION,1947.0,1.0,4.0,193788.0,37854.0,155934.0,Office,138672.000000,59.0


In [39]:
y.head()

,SiteEnergyUse(kBtu),TotalGHGEmissions
0,2953338.0,112.06
1,1212551.0,8.45
2,9898724.0,69.01
3,6525887.0,47.24
4,16760217.0,116.84


## Séparation du jeu de données d'entraînement et du jeu de données de test
Remarque : comme les hyperparamètres et le modèle seront séléctionnées par validation croisée, le jeu de données d'entraînement servira à la fois pour l'entraînement et la validation.

### Définition de la "graine" pour le générateur pseudo-aléatoire
Sélectionner cette graine nous permet d'opérer, lors des validations croisées imbriquées, sur les mêmes plis ("folds") pour chaque modèle.

In [40]:
# Set random integer (seed)
seed = 42

# Seed the generator of pseudo-random number.
np.random.seed(seed)

## Création d'un modèle naïf ("baseline")

## Choix des hyperparamètres et sélection du modèle
Nous procédons à une validation croisée pour sélectionner les hyperparamètres et le modèle.

Pour que le prétraitement soit réalisé sans fuite de données, nous utiliserons un « pipeline ».

### Définition du prétraitement différencié selon les colonnes

Pour éviter les "fuites de données", nous effectuerons les transformations de prétraitement dans un "pipe-line" intégré à la recherche sur grille avec validation croisée :
* imputation
* standardisation
* passage au log
* *target encoding*
* interaction and polynomial features

In [41]:
# Select numerical columns
numeric_features = list(X.select_dtypes(include='number').columns)
print("Numeric features:\n", numeric_features)

# Select categorical columns (ordered and non-ordered)
categorical_features = list(X.select_dtypes(include='category').columns)
print("\nCategorical features:\n", categorical_features)

Numeric features:
 ['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA', 'ENERGYSTARScore']

Categorical features:
 ['BuildingType', 'PrimaryPropertyType', 'Neighborhood', 'LargestPropertyUseType']


In [47]:
# Load libraries
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer

# Preprocessing pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=0)), # iterative imputation
    ('scaler', StandardScaler()), # standardization
     ])

# Preprocessing pipeline for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), # simple imputation
    ('target_encoder', TargetEncoder()) # target encoding
    ])

# Preprocessing column-wise using pipelines for numeric and categorical features
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
    ])

In [55]:
# Testing preprocessing pipeline
X_preprocessed = preprocessor.fit_transform(X, y)

In [57]:
X_preprocessed[5]

array([ 1.19472912e+00, -4.41353200e-02, -3.09537688e-01, -8.24898934e-02,
        3.56414564e-01, -1.44776908e-01, -1.18447792e-01,  3.40097633e-01,
        7.81636253e+06,  5.03300237e+06,  4.88402604e+06,  9.15739357e+06])

!!! Attention : le target encoding doit être effectué avant la séparation des variables explicatives et des étiquettes

### Définition du pipe-line
Nous définissons un pipe-line incluant tout le pré-traitement et un modèle de regression fantôche.

Les véritables modèles seront définis dans les grilles de recherche.

In [60]:
# Load libraries
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Create a pipeline
pipe = Pipeline([
    ("preprocess", preprocessor), # preprocessing steps
    ("regressor", LinearRegression()), # dummy regression model
])

In [82]:
# Testing generic pipeline
pipe.fit(X,y)

Pipeline(memory=None,
         steps=[('preprocess',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   IterativeImputer(add_indicator=False,
                                                                                    estimator=None,
                                                                                    imputation_order='ascending',
                                                                                    initial_strategy='mean',
                                                                                    max_iter=10,
                               

### Création des espaces de recherche
Pour des raisons de performances, nous choississons d'établir une grille de recherche distincte pour chacun des modèles.



In [100]:
from sklearn.linear_model import ElasticNet

# Create space of candidate values for models and hyperparameters
search_space = [{
    "regressor": [ElasticNet(max_iter=10000)], # elastic net regressor
    "regressor__alpha": np.logspace(0, 4, 10), # constant that multiplies the penalty terms
    "regressor__l1_ratio": [0, 0.5, 1] #  mixing parameter
}]

In [71]:
np.logspace(0, 4, 10)

array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04])

### Définition de la grille de recherche avec validation croisée (interne)

Remarque : nous conservons la métrique par défaut de scikit-learn pour les régression : le score R².

In [98]:
# Load libraries
from sklearn.model_selection import GridSearchCV

# Create grid search
gridsearch = GridSearchCV(
    pipe, # use of the pipeline (preprocessing + model)
    search_space, # choice of hyper-parameters to test
    cv=5, # numbers of folds
    n_jobs=-1, # use all available cores (parallelization)
    iid=False, # to avoid a DeprecationWarning
)

In [99]:
# Testing the GridSearchCV
best_model = gridsearch.fit(X, y)

# Getting the parameters of the best model
best_model.best_estimator_.get_params()['regressor']

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=10000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

### Méthodes spécifiques de sélection des hyper-paramètres
Scikit-learn propose des méthodes de sélection des hyper-paramètres spécifiques à certains algorithmes d'apprentissage automatique.

See also : https://scikit-learn.org/stable/modules/grid_search.html#model-specific-cross-validation

## Évaluation finale du modèle sélectionné par validation croisée (externe)
Nous évaluons le modèle sélectionné, avec les hyperparamètres sélectionnés sur le jeu de données d'entraînement.

Pour avoir une évaluation non-biaisée, nous utilisons une validation croisée imbriquée ("*nested cross validation*").

In [102]:
# Load libraries
from sklearn.model_selection import cross_val_score

# Conduct nested cross-validation and output the average score
cross_val_score(gridsearch, X, y, 
                cv=5, # default, but specified to silence warning
                scoring='r2', # default, but specified to silence warning
               ).mean()

KeyboardInterrupt: 

## Ré-entraînement final du modèle sur l'ensemble du jeu de données 
Nous ré-entraînons le modèle sélectionné, en utilisant les hyperparamètres sélectionnés par validation croisé sur l'ensemble du jeu de données pour lequel nous disposons des étiquettes cibles : le jeu de données d'entraînement et le jeu de données de test.

Remarque : peut-être fait automatiquement par la validation croisée ?

In [ ]:
final_model = 

## Impact des valeurs hors-normes ("*outliers*")
Précédemment nous avons identifiés les valeurs hors-normes. Nous allons tester si le modèle est plus performant lorsque l'on conserve ou que l'on élimine ces valeurs hors-normes.

### Détection et traitement des valeurs "hors normes"
Nous utilisons la méthode de l'ellipse, basée sur l'hypothèse que les données sont normalement distribuées.

Après la standardisation ?

In [42]:
def elliptic_outliers(dataframe, contamination=0.1):
    """This function returns outliers based on the EllpticEnvelope methode.
    It assumes that the (numerical) features are normally distributed
    and take the rate of outliers (contamination) as parameter."""

    # Load libraries
    from sklearn.covariance import EllipticEnvelope

    # Create detector
    outlier_detector = EllipticEnvelope(contamination=contamination)

    # Fit detector
    outlier_detector.fit(dataframe)

    # Predict outliers
    outlier_detector.predict(dataframe)

    # Define mask of predicted outliers
    mask = outlier_detector.predict(dataframe) == -1

    # Apply the mask to the data
    return dataframe[mask]

In [43]:
dataframe = data
contamination = 0.1

elliptic_outliers(dataframe, contamination)

ValueError: could not convert string to float: 'Multifamily LR (1-4)'

## Utilisation du meilleur modèle ré-entraîné pour la prédiction
Nous ne disposons que d'une seule ligne données sur laquelle utiliser notre modèle pour estimer la consommation d'énergie et l'émission de gaz à effets de serre.

### Élimination des colonnes d'étiquettes
Afin d'avoir un jeu de données ayant le même format que le jeu d'entraînement.

In [ ]:


production_set = production_set.drop(target_cols, axis=1)

### Transformation des données à évaluer avec les étapes de prétraitement

In [5]:
# Transforming data with preprocessing steps…


## Visualisation des effets de la taille du jeu de données d'entraînement
Nous étudions l'impact de la taille du jeu de données d'entraînement pour déterminer si notre modèle gagnerait en exactitude à être entraîné sur un nombre plus important de données.

## Importance des variables

# Évaluer l’intérêt de l’"ENERGY STAR Score"
Nous cherchons à évaluer l'intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par l'équipe.
* 'ENERGYSTARScore' : An EPA calculated 1-100 rating that assesses a property’s overall energy performance, based on national data to control for differences among climate, building uses, and operations. A score of 50 represents the national median.
* Voir aussi : https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what

In [1]:
# Feature importance


# Performance of the model without the 'ENERGYSTARScore'



## Sauvegarde du modèle
Nous sauvegardons le modèle dans un fichier pickle pour usage ultérieur.

Nous indiquons dans le nom du fichier la version de scikit-learn.

In [ ]:
# Load libraries
from sklearn.externals import joblib

# Get scikit-learn version
scikit_version = joblib.__version__

# Save model as pickle file
joblib.dump(model, "model_{version}.pkl".format(version=scikit_version))

### Chargement du modèle

In [ ]:
# Load model from file
regressor = joblib.load("model_0.22.pkl")

# TO DO LIST
* afficher les marges d'erreur des régressions
* afficher la variance des performances des modèles
* courbes pour les performances des modèles en fonction des hyper-paramètres
* intégrer les imputeurs à la pipeline